<a href="https://colab.research.google.com/github/ozturkcemal/SupplyChainAnalytics/blob/main/06_DistributionPlanning/SlickOilGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing library to read xlsx files
!pip install pandas openpyxl

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:


# Import libraries
import openpyxl
import pandas as pd
import numpy as np  # Import NumPy for array manipulation


# Get the file name of the uploaded file
file_name = list(uploaded.keys())[0]

# Load the workbook
workbook = openpyxl.load_workbook(file_name, data_only=True)

# Initialize a dictionary to store DataFrames for each named range
named_ranges_data = {}

# Loop through all defined names in the workbook
for name, defined_name in workbook.defined_names.items():
    destinations = list(defined_name.destinations)
    if not destinations:
        continue  # Skip if there are no destinations

    for sheet_name, cell_range in destinations:
        try:
            sheet = workbook[sheet_name]

            # If it's a single cell, handle it differently
            if ":" not in cell_range:  # Single cell (no colon in the range)
                cell_value = sheet[cell_range].value
                # Convert the single cell value into a DataFrame (1x1)
                df = pd.DataFrame([[cell_value]])

            else:
                # Retrieve the data from the specified range
                data = []
                for row in sheet[cell_range]:
                    data.append([cell.value for cell in row])

                # Convert to DataFrame
                df = pd.DataFrame(data)

            # Store the DataFrame with the named range as the key
            named_ranges_data[name] = df

        except Exception as e:
            print(f"Error processing range {name}: {e}")
            continue

# Create an array for each DataFrame, named after the DataFrame
for range_name, df in named_ranges_data.items():
    globals()[range_name] = np.array(df)  # Converts the DataFrame to a NumPy array

# Example of accessing one of the dynamically created arrays
for range_name in named_ranges_data:
    print(f"Array created for {range_name}:")
    print(globals()[range_name])
    print()


wells = list(range(wells.shape[0]))
print("List of wells:", wells)

# Create a list of indices from 0 to num_refineries - 1
refineries= list(range(refineries.shape[1]))

# Print the resulting list
print("List of refineries:", refineries)


In [19]:
#installing pyomo
!pip install -q pyomo

In [ ]:
#installing coin or
!apt-get install -y -qq coinor-cbc

In [20]:
#importing pyomo environment
import pyomo.environ as pyo

In [21]:
#creating a model object
model = pyo.ConcreteModel()

In [ ]:
#defining variables
model.x = pyo.Var(wells,refineries, within=pyo.NonNegativeReals)
model.y = pyo.Var(refineries, within=pyo.NonNegativeReals)
model.x.pprint()
model.y.pprint()

In [ ]:
# Remove any existing objective component named "obj"
if hasattr(model, 'obj'):
    model.del_component('obj')

#defining objective function
model.obj = pyo.Objective(
    expr=sum(refCost[0][j] * model.y[j] for j in refineries) +
         sum(transportCost[i][j] * model.x[i, j] for i in wells for j in refineries if transportCost[i][j]>0),
    sense=pyo.minimize
)
model.obj.pprint()

In [ ]:
# make sure refinery amount is less then the total amount coming from all wells
model.constraint_wellRef = pyo.ConstraintList()
for j in refineries:
    model.constraint_wellRef.add(sum(model.x[i, j] for i in wells) >= model.y[j])
model.constraint_wellRef.pprint()

In [ ]:
#make sure the demand is satisfied
model.constraint_demand = pyo.ConstraintList()
model.constraint_demand.add(sum(model.y[j] for j in refineries) >= demand[0][0])
model.constraint_demand.pprint()

In [ ]:
#modeling well  capacity constraint

if hasattr(model, 'constraint_wellCap'):
       model.del_component(model.constraint_wellCap)

model.constraint_wellCap = pyo.ConstraintList()
for i in wells:
    well_capacity = float(wellCap[i, 0])  # Extract the single element
    model.constraint_wellCap.add(sum(model.x[i, j] for j in refineries) <= well_capacity)
model.constraint_wellCap.pprint()

In [ ]:
#modeling refinery capacity constraint
model.constraint_refCap = pyo.ConstraintList()
for j in refineries:
    model.constraint_refCap.add(sum(model.x[i, j] for i in wells) <= refCap[0][j])
model.constraint_refCap.pprint()

In [ ]:
#make sure no transport from a well to a refinery if there is no path
model.constraint_forbidden= pyo.ConstraintList()
for i in wells:
  for j in refineries:
    if transportCost[i][j] == 0:
      model.constraint_forbidden.add(model.x[i,j] == 0)
model.constraint_forbidden.pprint()

In [ ]:
#connecting cbc solver and printing the model
opt = pyo.SolverFactory('cbc')
model.pprint()

In [83]:
#solve the model
opt_solution = opt.solve(model)

In [ ]:
# Print the values of the decision variables
#we are shifting the indices for presentation
print("\nWells to refineries:")
for i in wells:
    for j in refineries:
      if model.x[i, j].value > 0:
        print(f"well {i+1} to refinery {j+1}: {model.x[i, j].value}")

print("\nRefineries to the customer :")
for j in refineries:
    if model.y[j].value > 0:
      print(f"refinery {j+1}: {model.y[j].value}")

print(f"total cost of refinery is {sum(refCost[0][j] * model.y[j].value for j in refineries)}")
print(f"total cost of transport between wells to refineries {sum(transportCost[i][j] * model.x[i, j].value for i in wells for j in refineries)}")
print(f"Total cost of distribution plan : {model.obj()}")